# Deduplication workflow

## Load dataset

In [1]:
import recordlinkage as rl
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
from recordlinkage.datasets import load_febrl2

df, true_pairs = load_febrl2(return_links=True)
df = df.sort_index()

/Users/fjsj/workspace/deduplication-slides/venv/lib/python3.6/site-packages/recordlinkage/datasets/febrl.py:51: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  verify_integrity=False


In [3]:
df.head(1)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-0-org,annabelle,friswell,205,meares place,sunningdale farm,wildes meadow,7018,wa,19761129,9016980


In [4]:
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'].fillna('-'), format='%Y%m%d', errors='coerce')
df['date_of_birth_year'] = df['date_of_birth'].dt.year
df['full_name_address'] = (
    (' ' + df['given_name'].fillna(''))
    .str.cat(' ' + df['surname'].fillna('') + ' ')
    .str.cat(df['address_1'].fillna('') + ' ')
    .str.cat(df['address_2'].fillna('') + ' ')
    .str.cat(df['suburb'].fillna('') + ' ')
).replace({'': np.nan})

In [5]:
df.head(3)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,date_of_birth_year,full_name_address
rec_id,,,,,,,,,,,,
rec-0-org,annabelle,friswell,205,meares place,sunningdale farm,wildes meadow,7018,wa,1976-11-29,9016980,1976.0,annabelle friswell meares place sunningdale f...
rec-1-org,chloe,nappo,3,balonne street,brindabella specialist centre,carnegie,2068,wa,1994-02-01,8222350,1994.0,chloe nappo balonne street brindabella specia...
rec-10-org,lucy,crouch,202,goulburn street,sattwa park,cromer,2096,vic,1940-03-05,1800445,1940.0,lucy crouch goulburn street sattwa park cromer


“This data set contains 5000 records (4000 originals and 1000 duplicates), with a maximum of 5 duplicates based on one original record (and a poisson distribution of duplicate records). Distribution of duplicates: 19 originals records have 5 duplicate records 47 originals records have 4 duplicate records 107 originals records have 3 duplicate records 141 originals records have 2 duplicate records 114 originals records have 1 duplicate record 572 originals records have no duplicate record”  
https://recordlinkage.readthedocs.io/en/latest/ref-datasets.html#recordlinkage.datasets.load_febrl2

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, rec-0-org to rec-999-org
Data columns (total 12 columns):
given_name            4891 non-null object
surname               4936 non-null object
street_number         4777 non-null object
address_1             4891 non-null object
address_2             4431 non-null object
suburb                4950 non-null object
postcode              5000 non-null object
state                 4952 non-null object
date_of_birth         4880 non-null datetime64[ns]
soc_sec_id            5000 non-null object
date_of_birth_year    4880 non-null float64
full_name_address     5000 non-null object
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 507.8+ KB


In [7]:
list(true_pairs)[:5]

[('rec-712-dup-1', 'rec-712-dup-0'),
 ('rec-712-dup-2', 'rec-712-dup-0'),
 ('rec-712-dup-2', 'rec-712-dup-1'),
 ('rec-712-org', 'rec-712-dup-0'),
 ('rec-712-org', 'rec-712-dup-1')]

## LSH pre-blocking

In [8]:
from lsh import cache, minhash
from datasketch.lsh import _optimal_param

seeds = 900
threshold = 0.4
num_bands, band_size = _optimal_param(
    threshold=threshold, num_perm=seeds, false_positive_weight=0.5, false_negative_weight=0.5)
(num_bands, band_size)

(150, 6)

In [9]:
hasher = minhash.MinHasher(seeds=seeds, char_ngram=2, hashbytes=4)
lshcache = cache.Cache(num_bands=num_bands, hasher=hasher)

In [10]:
for rec_id, full_name in df['full_name_address'].items():
    if not pd.isnull(full_name):
        lshcache.add_doc(full_name, rec_id)

In [11]:
import itertools

def _list_to_pairs(l):
    return itertools.combinations(l, 2)

lsh_pairs = set()

for bin_n, b in enumerate(lshcache.bins):
    if bin_n % 5 == 0:
        print(f"Compute LSH candidate pairs progress: {bin_n}/{len(lshcache.bins)}")
    for b_id in list(b.keys()):
        bucked_ids = b[b_id]
        if len(bucked_ids) > 1:
            for pair in _list_to_pairs(sorted(bucked_ids)):
                lsh_pairs.add(pair)

print(f"Compute LSH candidate pairs DONE")

Compute LSH candidate pairs progress: 0/150
Compute LSH candidate pairs progress: 5/150
Compute LSH candidate pairs progress: 10/150
Compute LSH candidate pairs progress: 15/150
Compute LSH candidate pairs progress: 20/150
Compute LSH candidate pairs progress: 25/150
Compute LSH candidate pairs progress: 30/150
Compute LSH candidate pairs progress: 35/150
Compute LSH candidate pairs progress: 40/150
Compute LSH candidate pairs progress: 45/150
Compute LSH candidate pairs progress: 50/150
Compute LSH candidate pairs progress: 55/150
Compute LSH candidate pairs progress: 60/150
Compute LSH candidate pairs progress: 65/150
Compute LSH candidate pairs progress: 70/150
Compute LSH candidate pairs progress: 75/150
Compute LSH candidate pairs progress: 80/150
Compute LSH candidate pairs progress: 85/150
Compute LSH candidate pairs progress: 90/150
Compute LSH candidate pairs progress: 95/150
Compute LSH candidate pairs progress: 100/150
Compute LSH candidate pairs progress: 105/150
Compute LS

In [12]:
def ngrams_set(text_document, ngram_range):  # based on sklearn VectorizerMixin._char_ngrams
    text_len = len(text_document)
    min_n, max_n = ngram_range
    if min_n == 1:
        # no need to do any slicing for unigrams
        # iterate through the string
        ngrams = set(text_document)
        min_n += 1
    else:
        ngrams = set()

    # bind method outside of loop to reduce overhead
    ngrams_append = ngrams.add

    for n in range(min_n, min(max_n + 1, text_len + 1)):
        for i in range(text_len - n + 1):
            ngrams_append(text_document[i: i + n])
    return ngrams

In [13]:
def jaccard(a, b):
    if len(a) == 0 or len(b) == 0:
        return 0
    return len(a & b) / len(a | b)

In [14]:
ngrams_dict = {
    doc_id: ngrams_set(full_name, (2, 2))
    for doc_id, full_name in df['full_name_address'].items()
}

In [15]:
lsh_pairs_filtered = [(x, y) for (x, y) in lsh_pairs if jaccard(ngrams_dict[x], ngrams_dict[y]) > threshold]

print("len(lsh_pairs)", len(lsh_pairs))
print("len(lsh_pairs_filtered)", len(lsh_pairs_filtered))

len(lsh_pairs) 77613
len(lsh_pairs_filtered) 2748


In [16]:
len(rl.Index().block('surname').index(df))

/Users/fjsj/workspace/deduplication-slides/venv/lib/python3.6/site-packages/recordlinkage/index.py:155: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  verify_integrity=False)
/Users/fjsj/workspace/deduplication-slides/venv/lib/python3.6/site-packages/recordlinkage/base.py:288: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  pairs = pairs[pairs.labels[0] > pairs.labels[1]]


49781

In [17]:
lsh_G = nx.from_edgelist(lsh_pairs_filtered)
print("len(lsh_G)", len(lsh_G))
lsh_G_subgraphs = list(nx.connected_component_subgraphs(lsh_G, copy=False))
partition = {rec_id: partition_id
             for partition_id, subgraph in enumerate(lsh_G_subgraphs) for rec_id in subgraph}
print("len(partition.values())", max(partition.values()))

len(lsh_G) 2164
len(partition.values()) 558


In [18]:
import infomap

def infomap_communities(g):
    infomapWrapper = infomap.Infomap("--two-level --silent")
    i_to_node = dict(enumerate(g.nodes))
    node_to_i = {v: k for k, v in i_to_node.items()}
    for a, b, weight in g.edges(data='weight'):
        x, y = node_to_i[a], node_to_i[b]
        infomapWrapper.addLink(x, y, float(weight))

    infomapWrapper.run()
    partition = {
        i_to_node[k]: v
        for k, v in infomapWrapper.getModules().items()
    }
    return partition

In [19]:
lsh_G = nx.Graph()
lsh_G.add_weighted_edges_from(
    (x, y, jaccard(ngrams_dict[x], ngrams_dict[y])) for (x, y) in lsh_pairs_filtered
)
print("len(lsh_G)", len(lsh_G))
partition = infomap_communities(lsh_G)
print("len(partition.values())", max(partition.values()))

len(lsh_G) 2164
len(partition.values()) 595


In [20]:
lsh_pairs_filtered_partitioned = [
    (x, y) for (x, y, weight)
    in lsh_G.edges.data('weight') if partition[x] == partition[y]]
print("len(lsh_pairs_filtered_partitioned)", len(lsh_pairs_filtered_partitioned))

len(lsh_pairs_filtered_partitioned) 2699


In [21]:
df = df.assign(lsh_block=df.index.map(partition))
df[df['lsh_block'] == df.loc['rec-2460-org']['lsh_block']]

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,date_of_birth_year,full_name_address,lsh_block
rec_id,,,,,,,,,,,,,
rec-2460-dup-0,nicholas,hathaay,151,NaN,flemington markets,mill park,6011,nsw,1939-03-05,1796925,1939.0,nicholas hathaay flemington markets mill park,277.0
rec-2460-dup-2,nicholas,hathaway,74,NaN,flemington m zrkets,mill park,6011,nsw,1939-03-05,1796925,1939.0,nicholas hathaway flemington m zrkets mill p...,277.0
rec-2460-dup-3,nichokas,hathaway,74,NaN,flemingtonmarkets,mill park,6011,nsw,1939-03-05,1796925,1939.0,nichokas hathaway flemingtonmarkets mill park,277.0
rec-2460-dup-4,nicholas,hathqway,NaN,NaN,flemingto nmarkets,mill aprk,6011,nsw,1939-03-05,1796925,1939.0,nicholas hathqway flemingto nmarkets mill aprk,277.0
rec-2460-org,nicholas,hathaway,74,NaN,flemington markets,mill park,6011,nsw,1939-03-05,1796925,1939.0,nicholas hathaway flemington markets mill park,277.0


In [22]:
df.loc['rec-2460':'rec-2461']

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,date_of_birth_year,full_name_address,lsh_block
rec_id,,,,,,,,,,,,,
rec-2460-dup-0,nicholas,hathaay,151,NaN,flemington markets,mill park,6011,nsw,1939-03-05,1796925,1939.0,nicholas hathaay flemington markets mill park,277.0
rec-2460-dup-1,nicholas,emmanuel,74,NaN,laurelb ank,mill park,6011,nsw,1939-03-05,1796925,1939.0,nicholas emmanuel laurelb ank mill park,NaN
rec-2460-dup-2,nicholas,hathaway,74,NaN,flemington m zrkets,mill park,6011,nsw,1939-03-05,1796925,1939.0,nicholas hathaway flemington m zrkets mill p...,277.0
rec-2460-dup-3,nichokas,hathaway,74,NaN,flemingtonmarkets,mill park,6011,nsw,1939-03-05,1796925,1939.0,nichokas hathaway flemingtonmarkets mill park,277.0
rec-2460-dup-4,nicholas,hathqway,NaN,NaN,flemingto nmarkets,mill aprk,6011,nsw,1939-03-05,1796925,1939.0,nicholas hathqway flemingto nmarkets mill aprk,277.0
rec-2460-org,nicholas,hathaway,74,NaN,flemington markets,mill park,6011,nsw,1939-03-05,1796925,1939.0,nicholas hathaway flemington markets mill park,277.0


In [23]:
# len(rl.Index().block('state').block('date_of_birth_year').index(df))

In [24]:
# lsh_G = nx.from_edgelist(lsh_pairs_filtered)
# print("len(lsh_G)", len(lsh_G))
# lsh_G_subgraphs = list(nx.connected_component_subgraphs(lsh_G, copy=False))
# print("len(lsh_G_subgraphs)", len(lsh_G_subgraphs))

# rec_id_to_lsh_block_dict = {
#     rec_id: lsh_block
#     for lsh_block, rec_id_list in enumerate(lsh_G_subgraphs)
#     for rec_id in rec_id_list
# }

# df = df.assign(lsh_block=df.index.map(rec_id_to_lsh_block_dict)) 
# df[df['lsh_block'] == df.loc['rec-1684-org']['lsh_block']]

## Learn blocking rules

In [25]:
data_d = df.to_dict(orient='index')
for d in data_d.values():
    for k, v in d.items():
        if pd.isnull(v):
            d[k] = None
        elif hasattr(v, 'isoformat'):
            d[k] = v.isoformat()

In [26]:
data_d['rec-712-dup-2']

{'given_name': 'jac ob',
 'surname': 'lanyon',
 'street_number': '5',
 'address_1': 'milne cove',
 'address_2': None,
 'suburb': 'oatlands',
 'postcode': '2602',
 'state': 'vic',
 'date_of_birth': '1908-07-12T00:00:00',
 'soc_sec_id': '9497788',
 'date_of_birth_year': 1908.0,
 'full_name_address': ' jac ob lanyon milne cove  oatlands ',
 'lsh_block': 332.0}

In [27]:
data_d['rec-2303-org']

{'given_name': 'polly',
 'surname': 'noble',
 'street_number': '32',
 'address_1': 'carrodus street',
 'address_2': 'clanoc cottage',
 'suburb': 'port lincoln',
 'postcode': '6025',
 'state': 'nsw',
 'date_of_birth': None,
 'soc_sec_id': '3008610',
 'date_of_birth_year': None,
 'full_name_address': ' polly noble carrodus street clanoc cottage port lincoln ',
 'lsh_block': None}

In [28]:
[
    ({x: data_d[x]['full_name_address']},
     {y: data_d[y]['full_name_address']})
    for x, y in list(true_pairs)
    if data_d[x]['lsh_block'] != data_d[y]['lsh_block']
][:10]

[({'rec-2460-dup-1': ' nicholas emmanuel  laurelb ank mill park '},
  {'rec-2460-dup-2': ' nicholas hathaway  flemington m zrkets mill park '}),
 ({'rec-2460-dup-1': ' nicholas emmanuel  laurelb ank mill park '},
  {'rec-2460-dup-0': ' nicholas hathaay  flemington markets mill park '}),
 ({'rec-2460-dup-1': ' nicholas emmanuel  laurelb ank mill park '},
  {'rec-2460-org': ' nicholas hathaway  flemington markets mill park '}),
 ({'rec-2460-dup-1': ' nicholas emmanuel  laurelb ank mill park '},
  {'rec-2460-dup-3': ' nichokas hathaway  flemingtonmarkets mill park '}),
 ({'rec-2460-dup-4': ' nicholas hathqway  flemingto nmarkets mill aprk '},
  {'rec-2460-dup-1': ' nicholas emmanuel  laurelb ank mill park '}),
 ({'rec-3542-dup-3': ' kirra stubbs dampier crescent jodayfne auburn '},
  {'rec-3542-dup-1': ' kirr gaskin  jodayne auburn '}),
 ({'rec-3542-dup-3': ' kirra stubbs dampier crescent jodayfne auburn '},
  {'rec-3542-dup-2': ' kirra gaskni lawley xtreet jodayne auburn '}),
 ({'rec-354

Instead of MinHash LSH, MinHash LSH Ensemble to block by "containment": https://ekzhu.github.io/datasketch/lshensemble.html

In [29]:
import dedupe

fields = [
    {'field': 'given_name', 'type': 'ShortString', 'has missing': True},
    {'field': 'surname', 'type': 'ShortString', 'has missing': True},
    {'field': 'street_number', 'type': 'ShortString', 'has missing': True},
    {'field': 'address_1', 'type': 'ShortString', 'has missing': True},
    {'field': 'address_2', 'type': 'ShortString', 'has missing': True},
    {'field': 'suburb', 'type': 'ShortString', 'has missing': True},
    {'field': 'postcode', 'type': 'ShortString'},
    {'field': 'state', 'type': 'Exact', 'has missing': True},
    {'field': 'date_of_birth', 'type': 'DateTime', 'has missing': True},
#     {'field': 'soc_sec_id', 'type': 'ShortString'},
    {'field': 'lsh_block', 'type': 'Exact', 'has missing': True},
]

deduper = dedupe.Dedupe(fields)
deduper.sample(data_d, blocked_proportion=1)

INFO:dedupe.canopy_index:Removing stop word ce
INFO:dedupe.canopy_index:Removing stop word re


In [30]:
from dedupe.labeler import Sample

original_length = len(data_d)
sample_size = original_length
data = dedupe.core.index(data_d)
index_data = Sample(data, sample_size, original_length)
sampled_records = Sample(index_data, sample_size, original_length)

predicates = deduper.data_model.predicates(index_predicates=False, canopies=False)
block_learner = dedupe.training.DedupeBlockLearner(predicates, sampled_records, index_data)

In [31]:
dupes = [(data_d[x], data_d[y]) for x, y in true_pairs.values]
learned_preds = block_learner.learn(dupes, recall=0.99)
display(learned_preds)

INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sortedAcronym, postcode), SimplePredicate: (wholeFieldPredicate, lsh_block))
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, date_of_birth), SimplePredicate: (wholeFieldPredicate, state))


((SimplePredicate: (sortedAcronym, postcode),
  SimplePredicate: (wholeFieldPredicate, lsh_block)),
 (SimplePredicate: (wholeFieldPredicate, date_of_birth),
  SimplePredicate: (wholeFieldPredicate, state)))

In [32]:
import dedupe.blocking as blocking

deduper_blocked_pairs = set()

deduper.blocker = blocking.Blocker(learned_preds)
for block in deduper._blockedPairs(deduper._blockData(data_d)):
    for pair_data in block:
        deduper_blocked_pairs.add((pair_data[0][0], pair_data[1][0]))

In [33]:
deduper_blocked_pairs_set = set(tuple(sorted([a, b])) for a, b in deduper_blocked_pairs)
true_pairs_set = set(tuple(sorted([a, b])) for a, b in true_pairs)

blocking_true_positives = true_pairs_set & deduper_blocked_pairs_set
blocking_false_positives = deduper_blocked_pairs_set - true_pairs_set
blocking_false_negatives = true_pairs_set - deduper_blocked_pairs_set

print('blocking_true_positives total:', len(blocking_true_positives))
print('blocking_false_positives total:', len(blocking_false_positives))
print('blocking_false_negatives total:', len(blocking_false_negatives))
print()

print("Blocking Precision:")
print(len(blocking_true_positives) / (len(blocking_true_positives) + len(blocking_false_positives)))

print("Blocking Recall:")
print(len(blocking_true_positives) / (len(blocking_true_positives) + len(blocking_false_negatives)))

blocking_true_positives total: 1916
blocking_false_positives total: 537
blocking_false_negatives total: 18

Blocking Precision:
0.7810843864655523
Blocking Recall:
0.9906928645294726


In [34]:
# G = nx.Graph()
# G.add_nodes_from(df.index)
# G.add_edges_from(true_pairs)
# print("len(G)", len(G))
# G_subgraphs = list(nx.connected_component_subgraphs(G, copy=False))
# print("len(G_subgraphs)", len(G_subgraphs))

In [35]:
# rec_id_to_subgraph_id_dict = {
#     rec_id: subgraph_id
#     for subgraph_id, rec_id_list in enumerate(G_subgraphs)
#     for rec_id in rec_id_list
# }

# df = df.assign(subgraph=df.index.map(rec_id_to_subgraph_id_dict)) 
# df.head(3)

In [36]:
# import random

# test_ratio = 0.2
# available_subgraphs = list(df['subgraph'].unique())
# test_subgraphs = set()
# total_pairs_count = len(G.edges)
# test_pairs_count = 0

# while test_pairs_count < (total_pairs_count * test_ratio):
#     subgraph_id = random.choice(available_subgraphs)
#     test_subgraphs.add(subgraph_id)
#     available_subgraphs.remove(subgraph_id)
#     test_df = df[df['subgraph'].isin(test_subgraphs)]
#     test_pairs_count = len(test_df) * (len(test_df) - 1) / 2

# train_df = df[~df['subgraph'].isin(test_subgraphs)]

## Blocking

## Scoring

In [37]:
compare = rl.Compare()

compare.string(
    'given_name', 'given_name',
    method='jarowinkler', label='given_name')
compare.string(
    'surname', 'surname',
    method='jarowinkler', label='surname')
compare.numeric(
    'street_number', 'street_number',
    method='gauss', offset=10, scale=1, label='street_number')
compare.string(
    'address_1', 'address_1',
    method='jarowinkler', label='address_1')
compare.string(
    'address_2', 'address_2',
    method='jarowinkler', label='address_2')
compare.string(
    'suburb', 'suburb',
    method='jarowinkler', label='suburb')
compare.string(
    'postcode', 'postcode',
    method='jarowinkler', label='postcode')
compare.exact(
    'state', 'state',
    label='state')
compare.date(
    'date_of_birth', 'date_of_birth',
    method='gauss', offset=10, scale=1, label='date_of_birth')
compare.string(
    'soc_sec_id', 'soc_sec_id',
    method='damerau_levenshtein', label='soc_sec_id')

TypeError: __init__() got an unexpected keyword argument 'method'

## Train-test split

## Classification + Clustering

## Evaluation